# Display Trace

## Packages

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from tqdm.auto import tqdm

In [2]:
def scale(v, s=1):
    """Scale the vector of y \in R3,
    s is the  scaler,
    It computes the normalized vector of y is s = 1 as default.
    """
    return s * np.array(v) / np.linalg.norm(v)


def rotate(r, n, a):
    """Compute the rotated vector of r \in R3,
    n \in R3 is the rotate axis,
    a \in R is the radius.
    """
    d = np.cross(n, r)

    if all(d == 0):
        return r

    o = scale(scale(r, np.cos(a)) + scale(d, np.sin(a)))
    return o


class Point(object):
    """A point has its position and x-, y- and z- normalized coordinates"""

    def __init__(self, p=[0, 0, 0], nx=[1, 0, 0], ny=[0, 1, 0], nz=[0, 0, 1]):
        """
        Initialize the point,
        - p: The origin of the point;
        - nx: The normed x axis;
        - ny: The normed y axis;
        - nz: The normed z axis.
        """
        self.p = np.array(p).astype(np.float32)
        self.nx = scale(nx)
        self.ny = scale(ny)
        self.nz = scale(nz)
        pass

    def translate(self, dp):
        """Translate the point by p \in R3"""
        dpx = scale(self.nx, dp[0])
        dpy = scale(self.ny, dp[1])
        dpz = scale(self.nz, dp[2])
        self.p += dpx + dpy + dpz
        pass

    def rotate(self, ax, ay, az):
        """The point rotates around its origin, by 3 axis at ax, ay and az radius"""
        # self.nx = rotate(self.nx, self.nx, ax)
        self.ny = rotate(self.ny, self.nx, ax)
        self.nz = rotate(self.nz, self.nx, ax)

        self.nx = rotate(self.nx, self.ny, ay)
        # self.ny = rotate(self.ny, self.ny, ay)
        self.nz = rotate(self.nz, self.ny, ay)

        self.nx = rotate(self.nx, self.nz, az)
        self.ny = rotate(self.ny, self.nz, az)
        # self.nz = rotate(self.nz, self.nz, az)
        pass

    def snip(self):
        """Get a quick snip of the point on the current state"""
        # The p changes in-place, so use hard copy here.
        return [self.p.copy(), self.nx, self.ny, self.nz]


class MotionTrace(object):
    """Trace the motion and record every step"""

    def __init__(self):
        """Initialize"""
        # Dynamic point
        self.point = Point()

        # The point walks step-by-step,
        # the trace records its path.
        self.trace = []
        self.num_steps = 0

        # Reset the speed and angle
        self.dax = 0
        self.day = 0
        self.daz = 0
        self.dp = np.array((0.0, 0, 0))

        # Place the first point
        self.forward()
        pass

    def forward(self, ap=(0, 0, 0), aax=0, aay=0, aaz=0):
        """
        Forward a step,
        - ap: The accelerate of translating the position;
        - aax, aay, aaz: The angular accelerate of rotation along the x-, y- and z- axis.
        """

        def rad(x):
            return x / 180 * np.pi

        self.dax += aax
        self.day += aay
        self.daz += aaz
        self.dp += ap

        self.point.translate(self.dp)
        self.point.rotate(rad(self.dax), rad(self.day), rad(self.daz))
        self.trace.append(self.point.snip())

        self.num_steps += 1
        pass

    def trace_origin(self):
        """
        Trace the origins along the motion,
        it will return a matrix \in R^{n x 3},
        - n refers the steps, it equals to the self.num_samples;
        - 3 refers the x-, y- and z- coordinates.
        """
        return np.array([e[0] for e in self.trace])

    def trace_finish(self, length=500):
        """
        Trace the finish along the motion,
        the input
        - length refers the length of the arm/leg.
        the length grows at the negative direction of the z-axis.
        
        It will return a matrix \in R^{n x 3},
        - n refers the steps, it equals to the self.num_samples;
        - 3 refers the x-, y- and z- coordinates;
        """
        return np.array([e[0] - e[3] * length for e in self.trace])

## Read Parameters

In [3]:
folder = os.path.join(
    os.environ["SYNC"], r"MotionData\data\2nd-Development", "parsedData"
)

event_name = {
    "0-SLW": 1,
    "1-MLW": 2,
    "2-FLW": 3,
    "3-RD": 4,
    "4-SD": 5,
    "5-sit": 6,
    "6-stand": 7,
    "7-RA": 8,
    "8-SA": 9,
}

event_table = [(a, b) for a, b in event_name.items()]
print(event_table)

# Read and mark the Signal
X3s = []
ys = []

for name, label in event_name.items():
    with open(os.path.join(folder, f"{name}.npy"), "rb") as f:
        mat = np.load(f)
    X3s.append(mat)
    ys.append(np.zeros(mat.shape[0]) + label)

X3 = np.concatenate(X3s)
y = np.concatenate(ys)

y[y==2] = 1
y[y==3] = 1
# Simple Report
X3.shape, y.shape

[('0-SLW', 1), ('1-MLW', 2), ('2-FLW', 3), ('3-RD', 4), ('4-SD', 5), ('5-sit', 6), ('6-stand', 7), ('7-RA', 8), ('8-SA', 9)]


((31625, 20, 6), (31625,))

## Pattern Analysis

In [7]:
def mk_clf():
    clf = Pipeline([("scaler", StandardScaler()), ("svc", SVC(gamma="auto", class_weight='balanced'))])
    return clf

X = X3.reshape((len(X3), -1))

print(X.shape) 

skf = StratifiedKFold(n_splits=10)
p = y * 0

for train, test in tqdm(skf.split(y, y)):
    clf = mk_clf()
    clf.fit(X[train], y[train])
    p[test] = clf.predict(X[test])

print(metrics.classification_report(y_true=y, y_pred=p))

select = test
select.shape, X.shape

(31625, 120)


0it [00:00, ?it/s]

              precision    recall  f1-score   support

         1.0       0.93      0.78      0.85      9410
         4.0       0.76      0.96      0.85      3140
         5.0       0.97      0.90      0.94      4291
         6.0       0.91      0.89      0.90      3646
         7.0       0.87      0.96      0.91      3850
         8.0       0.68      0.90      0.78      2899
         9.0       0.97      0.93      0.95      4389

    accuracy                           0.88     31625
   macro avg       0.87      0.90      0.88     31625
weighted avg       0.89      0.88      0.88     31625



((3162,), (31625, 120))

In [8]:
with open(os.path.join(folder, "raw7-svc.npy"), "wb") as f:
    np.save(f, y)
    np.save(f, p)